In [1]:
from pathlib import Path
import torch
import torch.nn as nn
from torchvision import transforms
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from pyquaternion import Quaternion
import cv2
import pandas as pd

import time
from collections import deque, defaultdict

import habitat
from habitat.config import Config
from habitat.config.default import get_config
from habitat.core.env import Env
from habitat.sims.habitat_simulator.actions import HabitatSimActions

# from habitat_baselines.rl.ddppo.policy.resnet import ResNet, resnet50, se_resneXt50
# from habitat_baselines.rl.ppo import PointNavBaselinePolicy
from habitat_baselines.agents.ppo_agents import PPOAgent
from gym.spaces import Box, Dict, Discrete

root = Path('../frames')
transform_ = transforms.ToPILImage()

In [2]:
models = {
#     'rgb':   '/Users/nimit/Documents/robomaster/habitat/models/v2/gibson-2plus-mp3d-train-val-test-se-resneXt50-rgb.pth',
#     'depth': '/Users/nimit/Documents/robomaster/habitat/models/v2/gibson-2plus-resnet50.pth',
#     'depth': '/Users/nimit/Documents/robomaster/habitat/models/v2/gibson-4plus-resnet50.pth',
    'rgb':   '/Users/nimit/Documents/robomaster/habitat/models/v2/ppo/rgb.pth',
    'depth': '/Users/nimit/Documents/robomaster/habitat/models/v2/ppo/depth.pth'
}

configs = {
#     'rgb':   '/Users/nimit/Documents/robomaster/habitat/habitat2robomaster/rgb_test.yaml',
    'depth': '/Users/nimit/Documents/robomaster/habitat/habitat2robomaster/configs/pointgoal/ppo/train.yaml',
#     'depth': '/Users/nimit/Documents/robomaster/habitat/habitat-api/habitat_baselines/config/pointnav/ddppo_pointnav.yaml'
}

In [3]:
c = Config()

c.RESOLUTION       = 256
c.HIDDEN_SIZE      = 512
c.RANDOM_SEED      = 7

c.INPUT_TYPE       = 'depth'
c.MODEL_PATH       = models[c.INPUT_TYPE]
c.GOAL_SENSOR_UUID = 'pointgoal_with_gps_compass'

c.freeze()

env = Env(config=get_config(configs[c.INPUT_TYPE]))
agent = PPOAgent(c)

2020-04-17 05:28:54,375 Initializing dataset PointNav-v1
2020-04-17 05:28:54,546 initializing sim Sim-v0
I0417 05:28:56.493104 10187 simulator.py:142] Loaded navmesh /Users/nimit/Documents/robomaster/habitat/habitat-api/data/scene_datasets/habitat-test-scenes/skokloster-castle.navmesh
2020-04-17 05:28:56,497 Initializing task Nav-v0


In [4]:
agent.reset()
observations = env.reset()

k = 10
rgb_k  = np.zeros((k, 256, 256, 3), dtype=np.uint8)
meta_k = np.zeros((k, 2), dtype=np.float)

i = 0
while not env.episode_over:
    action = agent.act(observations)
    if i == k:
        break

    cv2.imshow('rgb', observations['rgb'])
    cv2.imshow('rgb', observations['depth'])
    cv2.waitKey(1)

    observations = env.step(action)
    rgb_k[i:,] = observations['rgb']
    meta_k[i:,] = observations['pointgoal_with_gps_compass']

    i += 1

metrics = env.get_metrics()
print(metrics)

cv2.destroyWindow('rgb')

{'distance_to_goal': 12.886344909667969, 'success': 0.0, 'spl': 0.0, 'collisions': {'count': 0, 'is_collision': False}}


## DDPPO

In [5]:
from pathlib import Path
import numpy as np
import torch

from gym.spaces import Box, Dict, Discrete

from habitat_baselines.rl.ddppo.policy.resnet_policy import PointNavResNetPolicy
from habitat_baselines.common.utils import batch_obs

# MODEL
observation_spaces, action_spaces = Dict({
#     'depth': Box(low=0, high=1, shape=(256, 256, 1), dtype=np.float32),
    'rgb': Box(low=0, high=255, shape=(256, 256, 3), dtype=np.uint8),
    'pointgoal_with_gps_compass': Box( 
        low=np.finfo(np.float32).min,
        high=np.finfo(np.float32).max,
        shape=(2,),
        dtype=np.float32,
    ),
}), Discrete(4)

actor_critic = PointNavResNetPolicy(
    observation_space=observation_spaces,
    action_space=action_spaces,
    hidden_size=512,
    rnn_type='LSTM',
    num_recurrent_layers=2,
    backbone='resnet50',
    goal_sensor_uuid='pointgoal_with_gps_compass',
    normalize_visual_inputs='rgb' in observation_spaces.spaces.keys()
)

# DATA
batch = { # move to GPU
#     'depth': torch.stack([torch.from_numpy(np.float64(observations['depth']))], dim=0).to(dtype=torch.float),
    'rgb': torch.stack([torch.from_numpy(np.float64(observations['rgb']))], dim=0).to(dtype=torch.float),
    'pointgoal_with_gps_compass': torch.stack([torch.from_numpy(np.float64(observations['pointgoal_with_gps_compass']))], dim=0).to(dtype=torch.float)
}


# TRAIN
test_recurrent_hidden_states = torch.zeros(actor_critic.net.num_recurrent_layers, 1, 512) # move to GPU
not_done_masks = torch.zeros(1, 1) # move to GPU
prev_actions = torch.zeros(1, 1, dtype=torch.long) # move to GPU

gt_action = torch.LongTensor([3])

optim = torch.optim.Adam(actor_critic.parameters())
criterion = torch.nn.CrossEntropyLoss()

for _ in range(10):
    optim.zero_grad()

    # NOTE: from habitat_baselines/agents/ppo_agents.py#137
    # must replace:
    # * self.test_recurrent_hidden_states
    # * self.prev_actions
    # * self.not_done_masks
    _, actions, action_log_probs, test_recurrent_hidden_states = actor_critic.act(
        batch,
        test_recurrent_hidden_states.detach(),
        prev_actions,
        not_done_masks,
        deterministic=False)
    #  Make masks not done till reset (end of episode) will be called
    not_done_masks = torch.ones(1, 1) # move to GPU
    prev_actions.copy_(actions)

    loss = criterion(actor_critic.prev_distribution.logits, gt_action)
    print(loss)

    loss.backward()
    optim.step()

tensor(1.3862, grad_fn=<NllLossBackward>)
tensor(1.3664, grad_fn=<NllLossBackward>)
tensor(1.2773, grad_fn=<NllLossBackward>)
tensor(1.0256, grad_fn=<NllLossBackward>)
tensor(0.5969, grad_fn=<NllLossBackward>)
tensor(0.2320, grad_fn=<NllLossBackward>)
tensor(0.0751, grad_fn=<NllLossBackward>)
tensor(0.0262, grad_fn=<NllLossBackward>)
tensor(0.0103, grad_fn=<NllLossBackward>)
tensor(0.0046, grad_fn=<NllLossBackward>)


In [12]:
from model import ConditionalStateEncoderImitation
net = ConditionalStateEncoderImitation()

action = torch.LongTensor([1, 2, 3, 1, 2, 3, 1, 2, 3, 1])

optim = torch.optim.Adam(net.parameters())
criterion = torch.nn.CrossEntropyLoss()

for i in range(0, 9, 2):
    rgb  = torch.Tensor(rgb_k[i:i+2]) #torch.stack([torch.from_numpy(np.float64(observations['rgb']))], dim=0).to(dtype=torch.float)
    meta = torch.Tensor(meta_k[i:i+2]) #torch.stack([torch.from_numpy(np.float64(observations['pointgoal_with_gps_compass']))], dim=0).to(dtype=torch.float)

    print(net.test_recurrent_hidden_states.shape)
    optim.zero_grad()
    
#     net.not_done_masks[0,0]=1.

    # NOTE: from habitat_baselines/agents/ppo_agents.py#137
    _action = net((rgb, meta))
    loss = criterion(_action, action[i:i+2])
    print(loss)

    loss.backward()
    optim.step()

torch.Size([4, 2, 512])
tensor(1.3862, grad_fn=<NllLossBackward>)
torch.Size([4, 2, 512])
tensor(1.3863, grad_fn=<NllLossBackward>)
torch.Size([4, 2, 512])
tensor(1.3879, grad_fn=<NllLossBackward>)
torch.Size([4, 2, 512])
tensor(1.3598, grad_fn=<NllLossBackward>)
torch.Size([4, 2, 512])
tensor(1.3315, grad_fn=<NllLossBackward>)
